## 基于GloVe词向量的文本分类
> 宋文彦 
> 21302010062

### 代码部分

#### 导入torch和其他所需要的库

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.vocab import GloVe
from torch.utils.data import DataLoader
from torchtext.transforms import VocabTransform, ToTensor
from torch.nn.utils.rnn import pad_sequence
from torchtext.vocab import vocab
from collections import Counter

/opt/anaconda3/envs/swy_py38/lib/python3.8/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/opt/anaconda3/envs/swy_py38/lib/python3.8/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/opt/anaconda3/envs/swy_py38/lib/python3.8/site-packages/torchtext/transforms.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABO

#### 加载SST数据集
提取SST中的训练集 `train_data` 、验证集 `valid_data` 和测试集 `test_data`。

In [36]:
from datasets import load_dataset

# 加载 SST 数据集
dataset = load_dataset('sst', 'default', trust_remote_code=True)

# 分割训练、验证、测试集
train_data = dataset['train']
valid_data = dataset['validation']
test_data = dataset['test']

print(f"Number of train samples: {len(train_data)}")
print(f"Number of test samples: {len(test_data)}")
print(f"Number of validation samples: {len(valid_data)}")

def round_labels(example):
    '''
    @description: 将标签值转换为 0, 0.25, 0.5, 0.75, 1
    @param: {example} 一个样本
    
    @return: {example} 处理后的样本
    '''
    label = example['label']  # 提取出 label 值
    if label < 0.2:
        example['label'] = 0  # 极负面
    elif label < 0.4:
        example['label'] = 0.25  # 负面
    elif label < 0.6:
        example['label'] = 0.5  # 中性
    elif label < 0.8:
        example['label'] = 0.75  # 正面
    else:
        example['label'] = 1  # 极正面
    return example

# 重新处理数据集的标签
train_data = train_data.map(round_labels)
valid_data = valid_data.map(round_labels)
test_data = test_data.map(round_labels)

print("Train dataset sample:", set(example['label'] for example in train_data))
print("Validation dataset sample:", set(example['label'] for example in valid_data))
print("Test dataset sample:", set(example['label'] for example in test_data))

Number of train samples: 8544
Number of test samples: 2210
Number of validation samples: 1101
Train dataset sample: {0.75, 1.0, 0.0, 0.5, 0.25}
Validation dataset sample: {0.75, 0.5, 0.25, 1.0, 0.0}
Test dataset sample: {0.5, 0.75, 0.0, 1.0, 0.25}


#### 构建文本处理器和标签处理器
将GloVe词向量传入文本处理器。

In [20]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

# 文本处理器
class TextProcessor:
    def __init__(self, vocab, max_length=40):
        '''
        @param vocab: 词汇表
        @param max_length: 文本最大长度
        '''
        self.vocab = vocab
        self.max_length = max_length

    def __call__(self, text):
        '''
        @param text: 文本
        
        @return token_ids: 文本的词汇表索引张量
        '''
        # 将文本分词并映射到词汇表索引
        tokens = text.lower().split()
        token_ids = [self.vocab[token] if token in self.vocab else 0 for token in tokens]
        return torch.tensor(token_ids[:self.max_length])

# 标签处理器：将标签转换为张量
def label_processor(label):
    return torch.tensor(int(label))

# 初始化 TextProcessor
glove = GloVe(name='6B', dim=100)       # 使用 GloVe 词向量
text_processor = TextProcessor(glove.stoi)  # 传入 GloVe 词汇表

# 批处理函数
def collate_fn(batch):
    '''
    @param batch: 一个 batch 的数据

    @return texts: 文本张量
    '''
    texts, labels = zip(*[(example['sentence'], example['label']) for example in batch])
    texts = [text_processor(text) for text in texts]
    texts = pad_sequence(texts, batch_first=True)
    labels = torch.tensor(labels, dtype=torch.long)
    return texts, labels

# 创建 DataLoader
batch_size = 16
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)


#### 定义 GloVe + Transfomer + Pooling + Classifier 的模型
基于Transformer和GloVe词向量的文本分类模型。模型的结构包含以下几部分：

1. 嵌入层（Embedding Layer）:
   - 使用 `nn.Embedding` 将输入的词汇索引映射到对应的词向量。
   - 使用预训练的 `GloVe` 词向量进行初始化，确保模型在开始训练时已经具备良好的词向量表示。
   - 词向量的权重不可学习，防止在训练过程中被更新。

2. `Transformer` 编码器（Transformer Encoder）:
   - 使用 `nn.TransformerEncoder` 对文本序列进行编码。`Transformer` 的编码器能够捕捉句子中的长距离依赖关系和上下文信息。
   - 该编码器由若干层 `TransformerEncoderLayer` 组成，每一层包含多头自注意力机制和前馈神经网络。
   - `embedding_dim` 表示输入的词向量维度，`nhead=2` 表示使用2个注意力头，`dim_feedforward=hidden_dim` 是前馈神经网络的隐藏层维度。

3. 池化层（Pooling Layer）:
   - 使用 `nn.AdaptiveMaxPool1d(1)` 对 `Transformer` 编码器的输出进行自适应最大池化。池化的作用是从序列的每个时间步中提取出最重要的信息，并减少数据的维度。
   - 池化后的输出形状被调整为 `(batch_size, hidden_dim)`，以便传入分类器进行处理。

4. 分类器（Classifier）:
   - 分类器由两层全连接网络组成：
     1. 第一层将池化后的向量映射到隐藏层大小 `hidden_dim`，并通过 `ReLU` 激活函数增加非线性。
     2. 第二层将隐藏层的输出映射到类别数 `num_classes`，最终输出每个类别的预测得分。

5. 前向传播（Forward Pass）:
   - 输入的文本张量经过嵌入层转换为词向量表示，再通过 `Transformer` 编码器进行特征提取，最后经过池化层和分类器，输出每个样本所属类别的概率。

In [21]:
class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, num_classes):
        '''
        @param vocab_size: 词汇表大小
        @param embedding_dim: 词向量维度
        @param hidden_dim: 隐藏层维度
        @param num_layers: TransformerEncoder 层数
        @param num_classes: 类别数量

        @return: None
        '''
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        self.embedding.weight.data.copy_(text_processor.vocab.vectors)
        self.embedding.weight.requires_grad = False

        self.transformer_encoder = nn.TransformerEncoder(nn.TransformerEncoderLayer(embedding_dim, nhead=2, dim_feedforward=hidden_dim), num_layers)
        
        #self.pooling = nn.AdaptiveAvgPool1d(1)
        self.pooling = nn.AdaptiveMaxPool1d(1)
        
        self.classifier = nn.Sequential(
            nn.Linear(embedding_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x) -> torch.Tensor:
        '''
        @param x: 输入文本张量，形状为 (batch_size, seq_len)

        @return: 输出类别张量，形状为 (batch_size, num_classes)
        '''
        embedded = self.embedding(x)
        encoded = self.transformer_encoder(embedded)
        pooled = self.pooling(encoded.permute(0, 2, 1))
        pooled = pooled.view(pooled.size(0), -1)
        output = self.classifier(pooled)
        return output

#### 定义 Random + RNN + Pooling + Classifier 的模型

定义基于RNN（循环神经网络）的文本分类模型，适用于自然语言处理任务中的文本分类问题。模型包含以下几个核心部分：

1. 词嵌入层（Embedding Layer）:
    - 使用nn.Embedding将输入的词汇索引映射为词向量。这一步通过将每个单词的索引转换为一个固定大小的向量表示，使得模型能够处理文本数据中的词汇信息。
    - 输入为词汇表的大小 `vocab_size`，输出为词向量的维度 `embedding_dim`。

2. RNN层（Recurrent Neural Network Layer）:
    - 使用 `nn.RNN` 来处理输入的词向量序列，并学习序列中单词的上下文信息。RNN通过循环的方式处理序列数据，适合捕捉句子中的时间依赖关系。
    - `embedding_dim` 为输入词向量的维度，`hidden_dim` 为RNN的隐藏状态维度，`num_layers` 表示堆叠RNN层的数量。

3. 池化层（Pooling Layer）:
    - 使用nn.AdaptiveAvgPool1d(1)进行自适应平均池化操作，将RNN层输出的序列压缩成一个固定大小的向量。这一步简化了时间维度，并保留了RNN输出中的关键信息。

4. 分类器（Classifier）:
    - 分类器由两个全连接层组成。首先，RNN输出的特征向量通过线性变换映射到隐藏层维度，并经过 `ReLU` 激活函数增加非线性。然后，进一步映射到目标类别数 `num_classes`，输出每个类别的预测概率

5. 前向传播（Forward Pass）:
    - 在前向传播过程中，输入的文本序列首先通过词嵌入层转换为词向量表示，再经过RNN处理序列信息。随后通过池化层提取全局特征，最后通过分类器输出预测结果。

In [22]:
class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, num_classes):
        '''
        @param: vocab_size: 词汇表大小
        @param: embedding_dim: 词向量维度
        @param: hidden_dim: 隐藏层维度
        @param: num_layers: RNN 层数
        @param: num_classes: 分类类别数

        @output: output: 模型输出   
        '''
        super(TextClassificationModel, self).__init__()
        
        # 词嵌入层：将输入的单词索引转换为对应的词向量表示
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # RNN层
        self.rnn = nn.RNN(embedding_dim, hidden_dim, num_layers, batch_first=True)

        # 自适应平均池化层：将RNN输出的时间步维度池化为一个固定大小
        self.pooling = nn.AdaptiveAvgPool1d(1)

        # 分类器：由两层全连接层组成
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x) -> torch.Tensor:
        embedded = self.embedding(x)    # 词嵌入
        rnn_output, _ = self.rnn(embedded)
        pooled = self.pooling(rnn_output.permute(0, 2, 1))
        pooled = pooled.view(pooled.size(0), -1)
        output = self.classifier(pooled)
        return output

#### 定义模型超参数，创建优化器和损失函数
1. 使用 `TextClassificationModel` 类创建一个文本分类模型对象。
   1. `vocab_size` 为GloVe的词汇表长度，即使用的单词数量；
   2. `embedding_dim` 为词向量的维度；
   3. `num_layers` 为编码器的层数；
   4. `num_classes` 为任务的类别数。
2. 创建优化器 `optimizer`，使用 Adam 优化算法更新模型参数，学习率为0.001。
3. 使用 `nn.CrossEntropyLoss` 创建一个交叉熵损失函数对象 `criterion`。

In [31]:
# 模型的超参数
vocab_size = len(glove.stoi)
embedding_dim = 100
hidden_dim = 128
num_layers = 2
num_classes = 5

# 创建文本分类模型的实例，传入词汇表大小、嵌入维度、隐藏层维度、编码器层数和类别数
model = TextClassificationModel(vocab_size, embedding_dim, hidden_dim, num_layers, num_classes)

# 创建优化器，定义损失函数
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()


#### 模型训练
* `optimizer.zero_grad()` 在每个批次开始时，将优化器的梯度缓冲区清零，确保每个批次的梯度计算是独立的。
* `loss = criterion(predictions, batch.label)` 计算预测结果与批次的标签之间的损失。
* `loss.backward()` 根据损失值，计算模型参数的梯度。
* `optimizer.step()` 根据优化器的更新规则，更新模型的参数，以减小损失函数的值。

In [32]:
# 模型训练函数
def train_model(model, iterator, optimizer, criterion):
    '''
    @param: model: 模型
    @param: iterator: 数据加载器
    @param: optimizer: 优化器

    @output: None
    '''
    model.train()
    for batch in iterator:
        texts, labels = batch  # 直接解包 tuple
        optimizer.zero_grad()
        predictions = model(texts)
        loss = criterion(predictions, labels)
        loss.backward()
        optimizer.step()

#### 模型验证
* 使用 `torch.no_grad()` 以禁用梯度计算，减少内存消耗和计算量。
* 返回在整个验证集上的损失和准确率。

In [33]:
# 模型验证函数
def evaluate_model(model, iterator, criterion) -> float:
    '''
    @param: model: 模型
    @param: iterator: 数据加载器
    @param: criterion: 损失函数

    @output: total_loss: 平均损失
    '''
    model.eval()
    total_loss = 0
    correct = 0
    with torch.no_grad():
        for batch in iterator:
            texts, labels = batch  # 直接解包 tuple
            predictions = model(texts)
            loss = criterion(predictions, labels)
            total_loss += loss.item()
            predicted_labels = predictions.argmax(1)
            correct += (predicted_labels == labels).sum().item()
    return total_loss / len(iterator), correct / len(iterator.dataset)


#### 模型训练
* 训练的总Epoch数为16。
* 调用已定义的 `train_model` 函数和 `evaluate_model` 函数。
* 在每个Epoch结束后，打印当前Epoch的验证集损失和准确率，监控模型在训练过程中的性能，并观察模型是否出现过拟合或欠拟合的情况。

In [34]:
# 训练模型
EPOCHS = 16
for epoch in range(EPOCHS):
    train_model(model, train_loader, optimizer, criterion)
    valid_loss, valid_acc = evaluate_model(model, valid_loader, criterion)
    print(f'Epoch: {epoch+1:02}\tValidation Loss: {valid_loss:.3f} | Validation Acc: {valid_acc*100:.2f}%')

/var/folders/d6/ncp146m53fz3rfc_06r2pj_40000gn/T/ipykernel_41418/3771617310.py:29: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  labels = torch.tensor(labels, dtype=torch.long)


Epoch: 01	Validation Loss: 0.412 | Validation Acc: 85.01%
Epoch: 02	Validation Loss: 0.388 | Validation Acc: 84.74%
Epoch: 03	Validation Loss: 0.392 | Validation Acc: 84.20%
Epoch: 04	Validation Loss: 0.406 | Validation Acc: 83.83%
Epoch: 05	Validation Loss: 0.429 | Validation Acc: 83.29%
Epoch: 06	Validation Loss: 0.536 | Validation Acc: 77.57%
Epoch: 07	Validation Loss: 0.539 | Validation Acc: 81.38%
Epoch: 08	Validation Loss: 0.889 | Validation Acc: 75.66%
Epoch: 09	Validation Loss: 0.799 | Validation Acc: 80.65%
Epoch: 10	Validation Loss: 0.802 | Validation Acc: 81.29%
Epoch: 11	Validation Loss: 1.134 | Validation Acc: 79.02%
Epoch: 12	Validation Loss: 1.131 | Validation Acc: 80.84%
Epoch: 13	Validation Loss: 0.957 | Validation Acc: 83.83%
Epoch: 14	Validation Loss: 1.182 | Validation Acc: 83.29%
Epoch: 15	Validation Loss: 1.321 | Validation Acc: 79.29%
Epoch: 16	Validation Loss: 1.273 | Validation Acc: 83.02%


#### 模型测试与评估

In [35]:
# Test the model
test_loss, test_acc = evaluate_model(model, test_loader, criterion)
print(f'Test Loss: {test_loss:.4f} | Test Acc: {test_acc*100:.2f}%')

/var/folders/d6/ncp146m53fz3rfc_06r2pj_40000gn/T/ipykernel_41418/3771617310.py:29: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  labels = torch.tensor(labels, dtype=torch.long)


Test Loss: 1.2790 | Test Acc: 81.58%
